In [ ]:
!rm -rf Pytorch_Modules

In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import DataLoader

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the Pytorch_Modules directory, download it from GitHub if it doesn't work
try:
    from Pytorch_Modules import engine, prediction, utils, alexnet
except:
    # Get the Pytorch_Modules scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/yasinahmadpoor/FromScratch
    !mv FromScratch/Pytorch_Modules .
    !rm -rf FromScratch
    from Pytorch_Modules import engine, prediction, utils


device = "cuda" if torch.cuda.is_available() else "cpu"

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'FromScratch'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 193 (delta 54), reused 52 (delta 28), pack-reused 106
Receiving objects: 100% (193/193), 2.08 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [3]:
def get_train_test_loader(data_dir,
                           batch_size,
                           augment,
                           shuffle=True):

    # define normalizer
    normalize = transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])    # Normalize a tensor image with mean and standard deviation for n channels, this transform will normalize each channel of the input.


    # define transforms
    valid_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),          # transforms.ToTensor() automatically scales the images to [0,1] range
                normalize])


    if augment:
            train_transform = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
    else:
            train_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),
                normalize,
            ])

    # download and create datasets
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, transform=train_transform, download=True)
    valid_dataset = datasets.CIFAR10(root=data_dir, train=False, transform=valid_transform, download=True)

    # define the data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = shuffle)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_dataset, valid_dataset, train_loader, valid_loader)

BATCH_SIZE = 64
train_data, test_data, train_dataloader, test_dataloader = get_train_test_loader(data_dir = 'CIFAR10_data',
                                                    batch_size = BATCH_SIZE,
                                                    augment = False)
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting CIFAR10_data/cifar-10-python.tar.gz to CIFAR10_data
Files already downloaded and verified
Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7fd5d1186460>, <torch.utils.data.dataloader.DataLoader object at 0x7fd5d116c8e0>)
Length of train dataloader: 782 batches of 64
Length of test dataloader: 157 batches of 64


In [4]:
# # Instantiate an instance of the model from the "model_builder.py" script
# torch.manual_seed(42)
# model = alexnet.AlexNet(input_channels=3,
#                         num_classes=10).to(device)

In [5]:


# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)


NUM_CLASSES = 10

# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.parameters():
  param.requires_grad = False


num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, NUM_CLASSES)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [6]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 227, 227), # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [32, 3, 227, 227]    [32, 10]             --                   Partial
├─Sequential (features)                  [32, 3, 227, 227]    [32, 256, 6, 6]      --                   False
│    └─Conv2d (0)                        [32, 3, 227, 227]    [32, 64, 56, 56]     (23,296)             False
│    └─ReLU (1)                          [32, 64, 56, 56]     [32, 64, 56, 56]     --                   --
│    └─MaxPool2d (2)                     [32, 64, 56, 56]     [32, 64, 27, 27]     --                   --
│    └─Conv2d (3)                        [32, 64, 27, 27]     [32, 192, 27, 27]    (307,392)            False
│    └─ReLU (4)                          [32, 192, 27, 27]    [32, 192, 27, 27]    --                   --
│    └─MaxPool2d (5)                     [32, 192, 27, 27]    [32, 192, 13, 13]    --                   --
│    └─Conv2d (6

In [7]:
LEARNING_RATE = 0.005
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=0.005)

In [8]:
# Start training with help from engine.py
NUM_EPOCHS = 5
result = engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

12:41:04 --- Epoch: 1 | train_loss: 0.9443 | train_acc: 70.8660 | test_loss: 0.7279 | test_acc: 76.9208


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

12:47:52 --- Epoch: 4 | train_loss: 0.7963 | train_acc: 75.3976 | test_loss: 0.7574 | test_acc: 76.2241


0it [00:00, ?it/s]

0it [00:00, ?it/s]

12:50:06 --- Epoch: 5 | train_loss: 0.7967 | train_acc: 75.2318 | test_loss: 0.7051 | test_acc: 77.4084


In [ ]:
# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")